In [13]:
import torch
from torchvision.datasets import CelebA

import os


os.environ['http_proxy'] = "http://127.0.0.1:58591" 
os.environ['https_proxy'] = "http://127.0.0.1:58591" 

In [18]:
# celeba_datasets = CelebA("../../../../../../../datasets", download=True)
device = torch.device("cuda")
(torch.cuda.memory_allocated(device) / (1024*1024*1024))

0.0